In [248]:
from collections import defaultdict
import copy

import pint
from pint import UnitRegistry
units = UnitRegistry()
Q_ = units.Quantity

import sigfig

import autoprotocol
import json
from autoprotocol.protocol import Protocol
from autoprotocol.container import Well, WellGroup, Container
from autoprotocol.container_type import ContainerType
from autoprotocol.unit import Unit

In [249]:
def verify_amount(amount, total=None, maximum=None):
    
    """"checks validity of quantity and unit input when adding/removing/transferring reagents
    if valid returns standardized amount w/units in proper Pint format"""
    
    #note: if amount is not numerical, error message is automatically raised in attempting to compare mag to 0
    #note: units will be automatically checked against unit registry 
        #eventually restrict input to moles or volume only??
        
    if amount <=0:
        raise ValueError("Cannot transfer a negative amount")
    
    elif total!=None: #specific to remove method only
        if total==0:
            raise ValueError("Cannot transfer from an empty container")
            
        elif amount>total:
            raise ValueError("Quantity to be transferred should be less than total quantity in container")
    
    elif maximum!=None: #if max volume of container is specified 
        if amount>maximum:
            raise ValueError("Quantity to be transferred exceeds maximum container volume")
    
    else:
        return amount
        
def round_amount(amount, sigfigs=3):
    
    """"" rounds amount (which must be in proper Pint format) to a default of 3 significant figures"""
    
    mag=sigfig.round(amount.magnitude, sigfigs)
    units=amount.units
    
    return(Q_(mag, units))

In [7]:
#container types and containers

type_well96 = autoprotocol.container_type.PCR96

well96= Container(id, type_well96, name='96 well plate')

type_well4 = ContainerType(name='4-well PCR plate', is_tube=False, well_count=4, well_depth_mm=None, well_volume_ul=Unit(160.0, 'microliter'), well_coating=None, sterile=None, cover_types=None, seal_types=['ultra-clear', 'foil'], capabilities=['liquid_handle', 'sangerseq', 'spin', 'thermocycle', 'incubate', 'gel_separate', 'gel_purify', 'seal', 'dispense'], shortname='4-pcr', col_count=2, dead_volume_ul=Unit(3, 'microliter'), safe_min_volume_ul=Unit(5, 'microliter'))

well4= Container(id, type_well4, name='4 well plate')

type_beaker = ContainerType(name='250 mL beaker', is_tube=True, well_count=1, well_depth_mm=None, well_volume_ul=Unit(250, 'milliliter'), well_coating=None, sterile=None, cover_types=None, seal_types=['ultra-clear', 'foil'], capabilities=['liquid_handle', 'sangerseq', 'spin', 'thermocycle', 'incubate', 'gel_separate', 'gel_purify', 'seal', 'dispense'], shortname='beaker', col_count=0, dead_volume_ul=Unit(3, 'microliter'), safe_min_volume_ul=Unit(5, 'microliter'))

beaker250=Container(id, type_beaker, name='250 mL beaker')

In [230]:
"""Classes modelling chemicals and substances
"""

"""
material - solid, liquid, or gas reagent from which we can make a mixture.
stored with its properties (molar mass and density would probably be useful) and phase (state of matter)
"""

class MaterialModel(): 
    
    def __init__(self, name, phase, properties=None): 
        
        self.name = name ##we should standardize the input format for names eventually
        
        self.properties = properties #optional... can include density, molar mass, etc
        
        # phase is a state of matter (solid, liquid, gas) ... 
        #to distinguish from the more common "state" of system at any given time
        self.phase = phase 
        
    def __repr__(self): 
        return f'{self.name}'
    
    def __hash__(self): 
        return hash(self.name)
    

class MixtureModel(): 
  
    def __init__(self, phase=None):
        """        
        You can add components to the mixture, but can't remove them
        (unless you remove a proportion of the mixture, assuming it is liquid and ideal)
        """
        
        self.phase= phase 
        #if we want to specify the phase of the overall mixture to define behavior/properties
        #e.g. if aqeuous, capable of liquid transfer
        #otherwise takes None as default
        
        self.materials= defaultdict(lambda: 0)
         
            #there is  a single dictionary for all the phases of matter, and the entire material model is stored
    #so you can index by phase of matter if you desire, via a loop across the keys
    #but it makes doing math with the dicitonaries much more concise
        
    def add(self, amounts):

            for mat, amount in amounts.items():
                a= verify_amount(amount)
                self.materials[mat] += a
                
 
    def remove(self, amount):
        
        #assumes ideal mixture and removes proportional amount of everything
        #eventually extraction can be incorporated 
        
        if self.phase!='aqueous': 
            raise TypeError(f"Method not supported for this type of mixture")
            #only works for aqueous mixtures
        
        else: 
            self.removed_mixture = defaultdict(lambda: 0)
        
            initial_conc = self.conc.copy()
            initial_vol = self.total_volume
            
            verify_amount(amount, initial_vol)
            
            for mat, val in self.materials.items():
                if mat.phase!='liquid': #solids and gases
                    self.removed_mixture[mat] += round_amount((amount * initial_conc[mat].to(units.molar)).to(units.mol))
                    self.materials[mat] -= round_amount((amount * initial_conc[mat].to(units.molar)).to(units.mol))
                elif mat.phase=='liquid':
                    self.removed_mixture[mat] += round_amount((val * amount/initial_vol.to_reduced_units()))
                    self.materials[mat] -= round_amount((val * amount/initial_vol.to_reduced_units()))
            return self.removed_mixture
        
    @property
    def total_measure(self):
        return sum(list(self.materials.values()))
        # adds up total amount (moles) of substance in the mixture 
   
        
    @property
    def total_volume(self):
        
        if self.phase!='aqueous': 
            raise TypeError(f"Invalid property for this type of mixture")
            #only meaningful for aqueous mixtures
        
        else: 
            vol=0
            for mat, amount in self.materials.items():
                if mat.phase=='liquid':
                    vol+= amount
            return round_amount(vol)

    @property
    def conc(self): 
        
        self.concentrations=defaultdict(lambda: 0)
        
        if self.phase=='aqueous': #aqueous solutions, return molarity
            for key, val in self.materials.items(): 
                if key.phase!='liquid':
                    self.concentrations[key] = round_amount(val/self.total_volume.to(units.liter))
    
            return self.concentrations
    
        else: #all other solutions, return mole fraction
    
            for key, val in self.materials.items(): 
                self.concentrations[key] = round_amount(val/self.total_measure)

            return self.concentrations
        
    def __repr__(self): 
        
        mat_names = [m.name for m in self.materials.keys()] 
        return f"Mixture of {', '.join(mat_names)}"
        

    def __len__(self):  #total number of components added
        return len(self.materials)

    def __hash__(self): 
        return hash(str(self.materials) + str(id(self)))
    
class ContainerModel(Container):
    
    def __init__(self, container_type, name, contents, temp = None): 
        
        Container.__init__(self, id, container_type, name)
        
        self.max_vol=container_type.well_volume_ul
        
        self.wells=Container.all_wells(self)
        
        if container_type.is_tube==False:
        
            for well in self.wells:
                well.index=well.humanize()
                well.contents=contents
                well.temp=temp
        else:
            
            self.contents=contents
            self.temp=temp    
    
    def __repr__(self): 
        return f'{self.name}'
    
    def __hash__(self): 
        return hash(self.name)
    
class Workspace():
    
    def __init__(self, containers): 
        
        self.containers=containers
        
    def state(self):
        current=[]
        for container in self.containers:
            if container.container_type.is_tube==True:
                current.append([container.name, container.contents.materials, container.temp])
                
            else:
        
                for well in container.all_wells():
                    current.append([well, well.contents.materials, well.temp])

        return current
            
    
    def __repr__(self): 
        
        conts = [c.name for c in self.containers] 
        return f"Workspace containing {', '.join(conts)}"
    
    def __hash__(self): 
        return hash(self.containers)
    
    
class Workflow():
    
    def __init__(self, workspace): 
        
        self.workspace=workspace
    
    def step(self, action):
        
        result = action.do()
        return self.workspace.state(), result
    
    
class HeatAction():
    
    def __init__(self, container, f_temp): 
        self.container = container
        self.f_temp = f_temp
        
    
    def do(self):
        
        self.container.temp = self.f_temp
        
        return f'Heated {self.container} to {self.f_temp}'

class VortexAction():
    
    def __init__(self, container, rpm, time): 
        self.container = container
        self.rpm = rpm
        self.time = time
        
    def do(self): 
        return f'Vortexed {self.container} at {self.rpm} rpm for {self.time}' 

        
class TransferAction():

    def __init__(self, container_from, container_to, amount, index=None): 
        
        self.container_from = container_from
        self.container_to = container_to
        self.amount = amount
        self.index=index
        
        self.current=defaultdict(lambda:0)
        
        if self.index!=None:
            for well in self.container_to.all_wells():
                self.current[well.index]= well.contents.materials.copy()

    def do(self):
        
        if self.index!=None: #if transferring into a well of a plate
            
            removed_mixture=self.container_from.contents.remove(self.amount)
            
            for well in self.container_to.all_wells():
                
                if well.index == self.index:
                    
                    well.contents.add(removed_mixture)
                    
                else:
                    
                    well.contents=MixtureModel(phase='aqueous')
                    well.contents.add(self.current[well.index])
                    
            return f'Transfered {self.amount} from {self.container_from} to {self.index} in {self.container_to}' 
            
        else: #if transferring between two independent containers, e.g. beaker to beaker
        
            removed_mixture=self.container_from.contents.remove(self.amount)
            self.container_to.contents.add(removed_mixture)
        
            return f'Transfered {self.amount} from {self.container_from} to {self.container_to}'

In [234]:
HCOOH = MaterialModel(name='formic acid', phase='liquid')
GBL = MaterialModel(name='GBL', phase= 'liquid')
PbI2 = MaterialModel(name='lead iodide', phase= 'solid')

stock = MixtureModel('aqueous')
stock.add({GBL: Q_(20, 'mL'), HCOOH: Q_(30, 'mL'), PbI2: Q_(0.2, 'mol')})

empty=MixtureModel('aqueous')

beaker=ContainerModel(container_type=type_beaker, name='beaker 1', contents= stock)

wells=ContainerModel(container_type=type_well4, name='plate 1', contents= empty)

In [236]:
action1=TransferAction(beaker, wells, Q_(1, 'mL'), index='A1')
Workflow(Workspace([beaker, wells])).step(action1)

([['beaker 1',
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 19.6 <Unit('milliliter')>,
                formic acid: 29.4 <Unit('milliliter')>,
                lead iodide: 0.196 <Unit('mole')>}),
   None],
  [Well(plate 1, A1, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 0.4 <Unit('milliliter')>,
                formic acid: 0.6 <Unit('milliliter')>,
                lead iodide: 0.004 <Unit('mole')>}),
   None],
  [Well(plate 1, A2, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {}),
   None],
  [Well(plate 1, B1, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {}),
   None],
  [Well(plate 1, B2, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {}),
   None]],
 'Transfered 1 milliliter from beaker 1 to A1 in plate 1')

In [237]:
action2 = TransferAction(beaker, wells, Q_(1, 'mL'), index='A2')
Workflow(Workspace([beaker, wells])).step(action2)

/opt/anaconda3/lib/python3.7/site-packages/sigfig/sigfig.py:570: UserWarning: warning: 3 significant figures requested from number with only 2 significant figures
  warn("warning: %d significant figures requested from number with only %d significant figures" % (given['sigfigs'], len(num.map)))
/opt/anaconda3/lib/python3.7/site-packages/sigfig/sigfig.py:570: UserWarning: warning: 3 significant figures requested from number with only 1 significant figures
  warn("warning: %d significant figures requested from number with only %d significant figures" % (given['sigfigs'], len(num.map)))


([['beaker 1',
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 19.200000000000003 <Unit('milliliter')>,
                formic acid: 28.799999999999997 <Unit('milliliter')>,
                lead iodide: 0.192 <Unit('mole')>}),
   None],
  [Well(plate 1, A1, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 0.4 <Unit('milliliter')>,
                formic acid: 0.6 <Unit('milliliter')>,
                lead iodide: 0.004 <Unit('mole')>}),
   None],
  [Well(plate 1, A2, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 0.4 <Unit('milliliter')>,
                formic acid: 0.6 <Unit('milliliter')>,
                lead iodide: 0.004 <Unit('mole')>}),
   None],
  [Well(plate 1, B1, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {}),
   None],
  [Well(plate 1, B2, None),
   def

In [238]:
action3 = TransferAction(beaker, wells, Q_(1, 'mL'), index='A1')
Workflow(Workspace([beaker, wells])).step(action3)

([['beaker 1',
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 18.800000000000004 <Unit('milliliter')>,
                formic acid: 28.199999999999996 <Unit('milliliter')>,
                lead iodide: 0.188 <Unit('mole')>}),
   None],
  [Well(plate 1, A1, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 0.8 <Unit('milliliter')>,
                formic acid: 1.2 <Unit('milliliter')>,
                lead iodide: 0.008 <Unit('mole')>}),
   None],
  [Well(plate 1, A2, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 0.4 <Unit('milliliter')>,
                formic acid: 0.6 <Unit('milliliter')>,
                lead iodide: 0.004 <Unit('mole')>}),
   None],
  [Well(plate 1, B1, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {}),
   None],
  [Well(plate 1, B2, None),
   def

In [241]:
for well in wells.all_wells():
    TransferAction(beaker, wells, Q_(1, 'mL'), index=well.index).do()

Workspace([beaker, wells]).state()

[['beaker 1',
  defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
              {GBL: 15.600000000000014 <Unit('milliliter')>,
               formic acid: 23.399999999999984 <Unit('milliliter')>,
               lead iodide: 0.15599999999999997 <Unit('mole')>}),
  None],
 [Well(plate 1, A1, None),
  defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
              {GBL: 1.6 <Unit('milliliter')>,
               formic acid: 2.4 <Unit('milliliter')>,
               lead iodide: 0.016 <Unit('mole')>}),
  None],
 [Well(plate 1, A2, None),
  defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
              {GBL: 1.2000000000000002 <Unit('milliliter')>,
               formic acid: 1.7999999999999998 <Unit('milliliter')>,
               lead iodide: 0.012 <Unit('mole')>}),
  None],
 [Well(plate 1, B1, None),
  defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
              {GBL: 0.8 <Unit('millilite

In [244]:
for well in wells.all_wells():
    HeatAction(wells.well(well.index), Q_(70, 'degC')).do()

Workspace([beaker, wells]).state()

[['beaker 1',
  defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
              {GBL: 15.600000000000014 <Unit('milliliter')>,
               formic acid: 23.399999999999984 <Unit('milliliter')>,
               lead iodide: 0.15599999999999997 <Unit('mole')>}),
  None],
 [Well(plate 1, A1, None),
  defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
              {GBL: 1.6 <Unit('milliliter')>,
               formic acid: 2.4 <Unit('milliliter')>,
               lead iodide: 0.016 <Unit('mole')>}),
  70 <Unit('degC')>],
 [Well(plate 1, A2, None),
  defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
              {GBL: 1.2000000000000002 <Unit('milliliter')>,
               formic acid: 1.7999999999999998 <Unit('milliliter')>,
               lead iodide: 0.012 <Unit('mole')>}),
  70 <Unit('degC')>],
 [Well(plate 1, B1, None),
  defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
              

In [245]:
action6 = HeatAction(wells.well('A1'), Q_(90, 'degC'))

Workflow(Workspace([beaker, wells])).step(action6)

([['beaker 1',
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 15.600000000000014 <Unit('milliliter')>,
                formic acid: 23.399999999999984 <Unit('milliliter')>,
                lead iodide: 0.15599999999999997 <Unit('mole')>}),
   None],
  [Well(plate 1, A1, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 1.6 <Unit('milliliter')>,
                formic acid: 2.4 <Unit('milliliter')>,
                lead iodide: 0.016 <Unit('mole')>}),
   90 <Unit('degC')>],
  [Well(plate 1, A2, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 1.2000000000000002 <Unit('milliliter')>,
                formic acid: 1.7999999999999998 <Unit('milliliter')>,
                lead iodide: 0.012 <Unit('mole')>}),
   70 <Unit('degC')>],
  [Well(plate 1, B1, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda

In [247]:
action7 = VortexAction(wells.well('A1'), 750, Q_(15, 'min'))

Workflow(Workspace([beaker, wells])).step(action7)

([['beaker 1',
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 15.600000000000014 <Unit('milliliter')>,
                formic acid: 23.399999999999984 <Unit('milliliter')>,
                lead iodide: 0.15599999999999997 <Unit('mole')>}),
   None],
  [Well(plate 1, A1, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 1.6 <Unit('milliliter')>,
                formic acid: 2.4 <Unit('milliliter')>,
                lead iodide: 0.016 <Unit('mole')>}),
   90 <Unit('degC')>],
  [Well(plate 1, A2, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda>()>,
               {GBL: 1.2000000000000002 <Unit('milliliter')>,
                formic acid: 1.7999999999999998 <Unit('milliliter')>,
                lead iodide: 0.012 <Unit('mole')>}),
   70 <Unit('degC')>],
  [Well(plate 1, B1, None),
   defaultdict(<function __main__.MixtureModel.__init__.<locals>.<lambda